# Phase Space Analysis of Algorithmic Stablecoins

This notebook visualizes the system dynamics in phase space to identify:

- **Attractors**: Stable equilibrium points
- **Repellers**: Unstable equilibrium points  
- **Trajectories**: System evolution under stress
- **Critical regions**: Areas of instability

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
sys.path.insert(0, '..')

from model import SystemParameters, simulate_step
from experiments import run_collateral_shock_experiment, run_liquidity_crisis_experiment

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 10)

## 1. Generate Trajectories for Phase Space

We run multiple simulations with different initial conditions and shock parameters.

In [ ]:
# Base parameters
params = SystemParameters(random_seed=42)

# Generate trajectories with different shock magnitudes
trajectories = []
shock_values = [-0.1, -0.2, -0.3, -0.4, -0.5]

for shock in shock_values:
    result = run_collateral_shock_experiment(
        shock_magnitude=shock,
        shock_time=100,
        params=params,
        n_steps=500
    )
    trajectories.append({
        'shock': shock,
        'price': result.price,
        'supply': result.supply,
        'liquidity': result.liquidity,
        'demand': result.demand
    })

print(f"Generated {len(trajectories)} trajectories")

## 2. 2D Phase Space: Price vs Supply

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Price vs Supply
colors = plt.cm.viridis(np.linspace(0, 1, len(trajectories)))
for i, traj in enumerate(trajectories):
    axes[0].plot(traj['supply'], traj['price'], 
                 color=colors[i], 
                 label=f"Shock: {traj['shock']:.0%}",
                 linewidth=2, alpha=0.7)
    # Mark starting point
    axes[0].scatter(traj['supply'][0], traj['price'][0], 
                    color=colors[i], s=100, marker='o', edgecolors='black')
    # Mark ending point
    axes[0].scatter(traj['supply'][-1], traj['price'][-1], 
                    color=colors[i], s=100, marker='s', edgecolors='black')

# Mark equilibrium
axes[0].axhline(y=1.0, color='red', linestyle='--', linewidth=2, label='Peg (P=1)')
axes[0].axhline(y=0.5, color='darkred', linestyle=':', linewidth=2, label='Collapse (P=0.5)')

axes[0].set_xlabel('Supply (S)', fontsize=12)
axes[0].set_ylabel('Price (P)', fontsize=12)
axes[0].set_title('Phase Space: Price vs Supply', fontsize=14)
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Price vs Liquidity
for i, traj in enumerate(trajectories):
    axes[1].plot(traj['liquidity'], traj['price'], 
                 color=colors[i], 
                 label=f"Shock: {traj['shock']:.0%}",
                 linewidth=2, alpha=0.7)
    axes[1].scatter(traj['liquidity'][0], traj['price'][0], 
                    color=colors[i], s=100, marker='o', edgecolors='black')
    axes[1].scatter(traj['liquidity'][-1], traj['price'][-1], 
                    color=colors[i], s=100, marker='s', edgecolors='black')

axes[1].axhline(y=1.0, color='red', linestyle='--', linewidth=2, label='Peg (P=1)')
axes[1].axhline(y=0.5, color='darkred', linestyle=':', linewidth=2, label='Collapse (P=0.5)')

axes[1].set_xlabel('Liquidity (L)', fontsize=12)
axes[1].set_ylabel('Price (P)', fontsize=12)
axes[1].set_title('Phase Space: Price vs Liquidity', fontsize=14)
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/plots/phase_space_2d.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. 3D Phase Space: Price, Supply, Liquidity

In [ ]:
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

colors = plt.cm.plasma(np.linspace(0, 1, len(trajectories)))

for i, traj in enumerate(trajectories):
    ax.plot3D(traj['supply'], traj['liquidity'], traj['price'],
              color=colors[i], linewidth=2, alpha=0.8,
              label=f"Shock: {traj['shock']:.0%}")
    
    # Mark start and end
    ax.scatter3D(traj['supply'][0], traj['liquidity'][0], traj['price'][0],
                 color=colors[i], s=100, marker='o')
    ax.scatter3D(traj['supply'][-1], traj['liquidity'][-1], traj['price'][-1],
                 color=colors[i], s=100, marker='^')

ax.set_xlabel('Supply (S)', fontsize=12)
ax.set_ylabel('Liquidity (L)', fontsize=12)
ax.set_zlabel('Price (P)', fontsize=12)
ax.set_title('3D Phase Space Trajectory', fontsize=14)
ax.legend(loc='upper left')

plt.tight_layout()
plt.savefig('../results/plots/phase_space_3d.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Vector Field Visualization

Visualize the direction of system evolution at different points in state space.

In [ ]:
# Create grid for vector field
S_range = np.linspace(5e5, 1.5e6, 15)
P_range = np.linspace(0.5, 1.5, 15)
S_grid, P_grid = np.meshgrid(S_range, P_range)

# Compute derivatives at each point
dS = np.zeros_like(S_grid)
dP = np.zeros_like(P_grid)

params = SystemParameters()
fixed_L = params.initial_liquidity
fixed_C = params.initial_collateral
fixed_D = params.initial_demand

for i in range(S_grid.shape[0]):
    for j in range(S_grid.shape[1]):
        S = S_grid[i, j]
        P = P_grid[i, j]
        
        # Simulate one step
        state = (S, P, fixed_C, fixed_L, fixed_D)
        new_state = simulate_step(state, params)
        
        dS[i, j] = new_state[0] - S
        dP[i, j] = new_state[1] - P

# Normalize arrows for visualization
magnitude = np.sqrt(dS**2 + dP**2)
dS_norm = dS / (magnitude + 1e-6)
dP_norm = dP / (magnitude + 1e-6)

# Plot vector field
fig, ax = plt.subplots(figsize=(12, 10))

# Color by magnitude
quiver = ax.quiver(S_grid, P_grid, dS_norm, dP_norm, magnitude,
                   cmap='coolwarm', alpha=0.8)
plt.colorbar(quiver, label='Magnitude of Change')

# Mark equilibrium line
ax.axhline(y=1.0, color='green', linestyle='--', linewidth=2, label='Peg (P=1)')
ax.axhline(y=0.5, color='red', linestyle='--', linewidth=2, label='Collapse Threshold')

# Overlay a trajectory
result = run_collateral_shock_experiment(
    shock_magnitude=-0.3,
    shock_time=100,
    params=params,
    n_steps=300
)
ax.plot(result.supply, result.price, 'k-', linewidth=3, label='Trajectory (-30% shock)')

ax.set_xlabel('Supply (S)', fontsize=12)
ax.set_ylabel('Price (P)', fontsize=12)
ax.set_title('Vector Field: System Dynamics in (S, P) Space', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/plots/vector_field.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Stability Regions

Identify regions of stability and instability based on system behavior.

In [ ]:
# Test stability across parameter space
liquidity_range = np.linspace(1e5, 2e6, 20)
shock_range = np.linspace(-0.6, -0.1, 20)

stability_map = np.zeros((len(shock_range), len(liquidity_range)))

for i, shock in enumerate(shock_range):
    for j, liq in enumerate(liquidity_range):
        test_params = SystemParameters(
            initial_liquidity=liq,
            liquidity_depth=liq
        )
        result = run_collateral_shock_experiment(
            shock_magnitude=shock,
            shock_time=50,
            params=test_params,
            n_steps=300
        )
        # 1 if stable (no collapse), 0 if collapsed
        stability_map[i, j] = 1 if result.time_to_collapse == np.inf else 0

# Plot stability map
fig, ax = plt.subplots(figsize=(12, 8))

im = ax.imshow(stability_map, aspect='auto', origin='lower',
               extent=[liquidity_range[0]/1e6, liquidity_range[-1]/1e6,
                       shock_range[0]*100, shock_range[-1]*100],
               cmap='RdYlGn')

ax.set_xlabel('Initial Liquidity (millions)', fontsize=12)
ax.set_ylabel('Shock Magnitude (%)', fontsize=12)
ax.set_title('Stability Map: Green = Stable, Red = Collapse', fontsize=14)

cbar = plt.colorbar(im)
cbar.set_label('Stability (1=Stable, 0=Collapse)')

plt.tight_layout()
plt.savefig('../results/plots/stability_regions.png', dpi=300, bbox_inches='tight')
plt.show()

# Calculate collapse probability by region
collapse_prob = 1 - stability_map.mean()
print(f"\nOverall collapse probability across tested parameter space: {collapse_prob:.1%}")

## 6. Key Findings

### Observations from Phase Space Analysis

1. **Attractor at P=1**: The system tends toward the peg under mild stress conditions

2. **Critical threshold**: Beyond a certain shock magnitude, trajectories diverge toward collapse

3. **Liquidity dependence**: Higher initial liquidity provides larger stability region

4. **Feedback amplification**: In the unstable region, small perturbations grow rapidly

In [ ]:
print("\n" + "=" * 60)
print("PHASE SPACE ANALYSIS COMPLETE")
print("=" * 60)
print("\nGenerated plots:")
print("  - phase_space_2d.png: 2D phase portraits")
print("  - phase_space_3d.png: 3D trajectory visualization")
print("  - vector_field.png: System dynamics vector field")
print("  - stability_regions.png: Parameter space stability map")